In [1]:
# !pip install selenium

  Using cached selenium-4.27.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.27.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.27.1-py3-none-any.whl (9.7 MB)
Using cached trio-0.27.0-py3-none-any.whl (481 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached PySocks-1.7.1-py3

In [4]:
# !pip install --upgrade selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 경로 설정
chrome_driver_path = r"C:\Users\carpe\OneDrive\Desktop\abc\chromedriver-win64\chromedriver.exe"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# 테스트로 Google 페이지 열기
driver.get("https://www.google.com")
print("ChromeDriver가 정상적으로 작동합니다!")
driver.quit()


ChromeDriver가 정상적으로 작동합니다!


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime, timedelta
import time

# ChromeDriver 경로 설정
chrome_driver_path = r"C:\Users\carpe\OneDrive\Desktop\abc\chromedriver-win64\chromedriver.exe"
service = webdriver.chrome.service.Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://at.agromarket.kr/domeinfo/sanRealtime.do"
driver.get(url)

# 데이터 저장을 위한 리스트
all_data = []
batch_size = 10  # 저장할 데이터 묶음 크기

# 날짜 범위 설정
start_date = datetime(2019, 1, 3)  # 시작 날짜
end_date = datetime(2024, 12, 20)  # 종료 날짜
current_date = start_date

# 날짜 반복
while current_date <= end_date:
    try:
        # JavaScript로 날짜 필드 값 설정
        date_script = f"document.getElementById('saledate').value = '{current_date.strftime('%Y-%m-%d')}'"
        driver.execute_script(date_script)
        print(f"날짜를 {current_date.strftime('%Y-%m-%d')}로 설정했습니다.")

        # 검색 버튼 클릭
        wait = WebDriverWait(driver, 10)
        search_button = wait.until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        search_button.click()

        # 데이터 로드 대기
        time.sleep(3)

        # 테이블 데이터 가져오기
        table = driver.find_element(By.TAG_NAME, "table")
        rows = table.find_elements(By.TAG_NAME, "tr")

        # 컬럼 개수 확인 및 동적 처리
        if rows:
            first_row = rows[1].find_elements(By.TAG_NAME, "td")
            column_count = len(first_row)
            print(f"컬럼 개수: {column_count}")
            columns = ["날짜"] + [f"컬럼{i+1}" for i in range(column_count)]  # 동적으로 컬럼 이름 생성

        # 현재 날짜 데이터 저장
        for row in rows[1:]:  # 첫 번째 행은 헤더로 가정
            cols = row.find_elements(By.TAG_NAME, "td")
            all_data.append([current_date.strftime("%Y-%m-%d")] + [col.text.strip() for col in cols])

        print(f"날짜 {current_date.strftime('%Y-%m-%d')} 데이터 수집 완료")

        # 10개씩 저장
        if len(all_data) >= batch_size:
            df = pd.DataFrame(all_data, columns=columns)
            # 파일에 추가 저장 (append 모드)
            df.to_csv("../data/raw/auction-data", index=False, mode="a", header=False, encoding="utf-8-sig")
            all_data = []  # 저장 후 리스트 초기화
            print("10개 데이터를 저장했습니다.")

    except Exception as e:
        print(f"날짜 {current_date.strftime('%Y-%m-%d')} 처리 중 오류 발생: {e}")

    # 다음 날짜로 이동
    current_date += timedelta(days=1)

# 남은 데이터 저장 (10개 미만일 경우)
if all_data:
    df = pd.DataFrame(all_data, columns=columns)
    df.to_csv("../data/raw/auction-data", index=False, mode="a", header=False, encoding="utf-8-sig")
    print(f"남은 {len(all_data)}개의 데이터를 저장했습니다.")

# 브라우저 종료
driver.quit()


날짜를 2019-01-03로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-03 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-04로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-04 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-05로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-05 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-06로 설정했습니다.
컬럼 개수: 1
날짜 2019-01-06 데이터 수집 완료
날짜를 2019-01-07로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-07 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-08로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-08 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-09로 설정했습니다.
컬럼 개수: 1
날짜 2019-01-09 데이터 수집 완료
날짜를 2019-01-10로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-10 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-11로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-11 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-12로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-12 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-13로 설정했습니다.
컬럼 개수: 1
날짜 2019-01-13 데이터 수집 완료
날짜를 2019-01-14로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-14 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-15로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-15 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 2019-01-16로 설정했습니다.
컬럼 개수: 10
날짜 2019-01-16 데이터 수집 완료
10개 데이터를 저장했습니다.
날짜를 

KeyboardInterrupt: 